<a href="https://colab.research.google.com/github/hatttruong/machine-learning-from-scratch/blob/master/explore_gradient_boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I implement some functions to understanding more about gradient boosting based on the blog explained.ai

Ref: https://explained.ai/gradient-boosting/index.html

# Gradient boosting regression by example

### Regression tree stump